In [6]:
from backtesting.test import SMA

In [7]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 9
    n2 = 26
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [8]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(GOOG, SmaCross, commission=.002)

bt.run()
bt.plot()

In [18]:
bt.run()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                      46.648
Equity Final [$]                      56818.7
Equity Peak [$]                       58856.5
Return [%]                            468.187
Buy & Hold Return [%]                 703.458
Max. Drawdown [%]                    -25.7741
Avg. Drawdown [%]                    -3.42082
Max. Drawdown Duration      654 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                   38
Win Rate [%]                          57.8947
Best Trade [%]                        51.3069
Worst Trade [%]                      -7.88788
Avg. Trade [%]                        6.62743
Max. Trade Duration         184 days 00:00:00
Avg. Trade Duration          46 days 00:00:00
Profit Factor                         5.39875
Expectancy [%]                        11.0615
SQN                               

In [14]:
#RSI BACKTEST
